In [50]:
import torch
import torchvision
import os
import numpy as np
from PIL import Image
from torchvision import transforms

In [36]:
def get_shuffled_images(data_dir):
    classes = os.listdir(data_dir)[1:]
    data = []
    for class_name in classes:
        class_dir = os.path.join(data_dir, class_name)
        class_label = 1 if class_name == 'Malignant' else 0
        for img_name in os.listdir(class_dir):
            image_path = os.path.join(class_dir, img_name)
            image = np.array(Image.open(image_path))
            label = class_label
            data.append((image, label))
    np.random.shuffle(data)
    return data

In [37]:
data_train_dir = "../data/train"
data_test_dir = "../data/test"

In [38]:
train_data = get_shuffled_images(data_train_dir)
test_data = get_shuffled_images(data_test_dir)

In [39]:
for image, label in train_data[:15]:
    print(f"label: {label}")

label: 1
label: 0
label: 1
label: 0
label: 1
label: 1
label: 1
label: 1
label: 0
label: 0
label: 0
label: 1
label: 0
label: 0
label: 1


In [51]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [52]:
transformed_train_data = [(transform(image), torch.tensor(label)) for image, label in train_data]
transformed_test_data = [(transform(image), torch.tensor(label)) for image, label in test_data]

In [56]:
train_images = torch.stack([item[0] for item in transformed_train_data])
train_labels = torch.stack([item[1] for item in transformed_train_data])
test_images = torch.stack([item[0] for item in transformed_test_data])
test_labels = torch.stack([item[1] for item in transformed_test_data])